In [23]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

# Veri seti boyutu
n_samples = 1000

# Veri seti özellikleri
property_types = ["oda", "daire", "villa"]
locations = ["şehir merkezi", "kırsal"]
pool_options = [0, 1]  # 0: Yok, 1: Var
wifi_options = [0, 1]  # 0: Yok, 1: Var
breakfast_options = [0, 1]  # 0: Yok, 1: Var

In [24]:
# Veri seti oluşturma
random.seed(42)
np.random.seed(42)

data = {
    "Ev Tipi": random.choices(property_types, k=n_samples),
    "Konum": random.choices(locations, k=n_samples),
    "Oda Sayısı": np.random.randint(1, 6, size=n_samples),
    "Kişi Sayısı": np.random.randint(1, 10, size=n_samples),
    "Havuz": random.choices(pool_options, k=n_samples),
    "WiFi": random.choices(wifi_options, k=n_samples),
    "Kahvaltı": random.choices(breakfast_options, k=n_samples),
    "Günlük Fiyat": np.random.randint(100, 1000, size=n_samples),
}

# DataFrame'e dönüştürme
df = pd.DataFrame(data)

# Veri setinin ilk 5 satırını ekrana yazdırma
print("Veri setinin ilk 5 satırı:")
print(df.head())

Veri setinin ilk 5 satırı:
  Ev Tipi          Konum  Oda Sayısı  Kişi Sayısı  Havuz  WiFi  Kahvaltı  \
0   daire  şehir merkezi           4            4      0     1         0   
1     oda         kırsal           5            4      1     1         0   
2     oda         kırsal           3            5      0     1         0   
3     oda         kırsal           5            8      0     0         1   
4   villa  şehir merkezi           5            5      0     0         1   

   Günlük Fiyat  
0           141  
1           777  
2           693  
3           528  
4           902  


In [25]:
# Kategorik verileri one-hot encoding ile sayısal hale getirme
if "Ev Tipi" in df.columns and "Konum" in df.columns:
    df = pd.get_dummies(df, columns=["Ev Tipi", "Konum"], drop_first=True)
else:
    print("Hata: Kategorik kolonlar bulunamadı!")

# Özellikler (X) ve hedef sütunu (Y) ayırma
if "Günlük Fiyat" in df.columns:
    X = df.drop("Günlük Fiyat", axis=1)
    y = df["Günlük Fiyat"]
else:
    print("Hata: Hedef kolon (Günlük Fiyat) bulunamadı!")
    X = pd.DataFrame()
    y = pd.Series()


In [ ]:
# Eğitim ve test setlerine ayırma
if not X.empty and not y.empty:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Eğitim ve test setlerinin boyutlarını ekrana yazdırma
    print("\nEğitim ve test setlerinin boyutları:")
    print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")
    
    print("\nVeri ön işleme tamamlandı ve veriler kaydedildi!")
else:
    print("Hata: Eğitim ve test setleri oluşturulamadı!")


Eğitim ve test setlerinin boyutları:
X_train: (800, 8), X_test: (200, 8)
y_train: (800,), y_test: (200,)
